In [0]:
import collections
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
#import seaborn as sns
#sns.set()


In [0]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris() # IRIS 데이터를 로딩
X = iris.data[:, [2,3]] # 0~3번 칼럼중에 2,3번 칼럼만 이용하는 것임.
Y = iris.target

In [0]:
## 150개 랜덤하게 뒤섞은 후에 학습데이터 100개, 테스트데이터 50개로 나눔.
NUM = 100
select = np.random.permutation(150) # 0~149 숫자들을 랜덤하게 뒤섞음
Xtr, Ytr = X[select[:NUM]], Y[select[:NUM]] # 100개  (0~99)
Xte, Yte = X[select[NUM:]], Y[select[NUM:]] #  50개  (100~149)  
print(Xtr.shape)
print(Xte.shape)

## k-NN(Nearest Neighbor)를 직접 구현해보자


k-NN의 아이디어는 간단하다. 
어떤 새로운 데이터  new_x에 대하여, 기존의 데이터(학습 데이터)에서 new_x와 가장 가까운 이웃들 k개를 찾는다. 
이 이웃들이 무슨 y값을 가지는지에 따라 new_x의 new_y값이 결정된다.



In [0]:
# Euclidean Distance Caculator
def dist(a, b):
    return np.linalg.norm(a - b) # default norm: norm 2
  
def dist_many(a, b, ax=1):
    return np.linalg.norm(a - b, axis=ax) # default norm: norm 2

In [0]:
class MyKnn(object):
  def __init__(self):
    pass
  
  def fit(self, x, y, k):
    '''
    k-NN의 학습은 모든 데이터 자체를 그대로 기억(저장)하는 것이다.
    '''
    self.X = x.copy()
    self.Y = y.copy()
    self.K = k
  
  def predict_one(self, new_x): 
    '''
    한개의 새로운 x에 대하여, 기존의 데이터(학습 데이터)에서 x와 가장 가까운 이웃들을 찾는다. 
    이 이웃들이 무슨 y값을 가지는지에 따라 new_x의 new_y값이 결정된다.
    '''
    distance = [dist(x, new_x) for x in self.X]
    d_and_y = zip(distance, self.X, self.Y)
    print('== Distance, X, Y')
    for v in d_and_y:
      print(v)
    
    
    d_and_y = zip(distance, self.X, self.Y)
    ''' 거리순으로 sort '''
    rank = sorted(d_and_y, key=lambda x: x[0])
    print('== Sorted')
    for v in rank[:self.K]:
      print(v)
    
    ''' y값 중에 어느 값이 제일 많은지 확인. 다수결로 정함. '''
    new_y = [v[2] for v in rank[:self.K]]
    print('new_y', new_y)
    cnter = collections.Counter(new_y)
    print(cnter.most_common(1)) # value & counter
    # 가장 빈도가 높은 y값
    y = cnter.most_common(1)[0][0]
    print('최종 y', y)
    return y

  def predict(self, xlist): 
    '''
    여러 x들에 대해 k-NN 수행
    '''
    ylist = []
    for new_x in xlist:
      distance = [np.linalg.norm(x-new_x, 2) for x in self.X]
      d_and_y = zip(distance, self.X, self.Y)
      rank = sorted(d_and_y, key=lambda x: x[0])
      new_y = [v[2] for v in rank[:self.K]]
      cnter = collections.Counter(new_y)
      y = cnter.most_common(1)[0][0]
      ylist.append(y)
    return np.array(ylist)
  
  

## k-NN동작 과정 출력하기

In [0]:
knn = MyKnn()
knn.fit(Xtr[:10], Ytr[:10], 3)
knn.predict_one(Xte[0])

## k-NN 학습

In [0]:
COLORMAP = {0: 'g', 1: 'r', 2: 'b'}
def mycolor(data):
  return [COLORMAP[v] for v in data]

In [0]:
knn = MyKnn()
knn.fit(Xtr, Ytr, 10) 

### k-NN classification 정확도 측정 - 테스트 데이터로 (Xte)

In [0]:
Ypr = knn.predict(Xte)
print(Ypr[:10])

### 예측값 (Ypr) 그려보기


In [0]:
plt.scatter(Xte[:, 0], Xte[:, 1], c=mycolor(Ypr))

### 정답(Yte) 그리기

In [0]:
plt.scatter(Xte[:, 0], Xte[:, 1], c=mycolor(Yte))

### 예측값(Ypr), 정답값(Yte) 비교

In [0]:
is_correct = (Ypr == Yte)
correct_cnt = np.sum(is_correct)
total = len(Yte)
wront_cnt = total - correct_cnt
print('정답: %d   오답: %d' % (correct_cnt, wront_cnt ))
myscore = correct_cnt/float(total)
print('마이 스코어 %.2f' % myscore)

### 예측이 틀린 것 찾기

In [0]:
# 맞춘 것은 'g'(Green), 틀린 것은 'r' (Red)
colors = ['g' if c else 'r' for c in is_correct] 
plt.scatter(Xte[:, 0], Xte[:, 1], c=colors)